### Download data from https://www.kaggle.com/competitions/jpx-tokyo-stock-exchange-prediction/data

In [1]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from lightgbm import LGBMRegressor
from datetime import datetime, date, timedelta
import matplotlib.pyplot as plt

In [2]:
train_prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv")

In [3]:
date_list = train_prices['Date'].unique()
stock_date = train_prices.groupby("Date")

In [4]:
rank_only = pd.DataFrame(columns=['Rank'])

for date_price in date_list:
    tmp = stock_date.get_group(date_price)
    tmp['Rank'] = tmp['Target'].rank(ascending=False, method='first') - 1
    rank_only = rank_only.append(tmp['Rank'].to_frame())
# print(rank_only)

train_prices['Rank'] = rank_only['Rank']
train_prices

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,Rank
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730,898.0
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324,325.0
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154,567.0
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053,364.0
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026,750.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816,580.0
2332527,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478,1119.0
2332528,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302,1941.0
2332529,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False,0.009098,1768.0


In [5]:
code_group = train_prices.groupby('SecuritiesCode')
code_list = list(code_group.groups.keys())

In [6]:
cols = ["Date", "SecuritiesCode", "Open", "High", "Low", "Close", "Volume", "Target"]

df = pd.DataFrame(columns=cols)
print(df)

for code in code_list:
#     print(code)
    tmp = code_group.get_group(code)
    
    tmp['Open'].fillna(value=tmp['Open'].mean(), inplace=True)
    tmp['High'].fillna(value=tmp['High'].mean(), inplace=True)
    tmp['Low'].fillna(value=tmp['Low'].mean(), inplace=True)
    tmp['Close'].fillna(value=tmp['Close'].mean(), inplace=True)
    tmp['Target'].fillna(value=tmp['Target'].mean(), inplace=True)
    tmp['Rank'].fillna(value=tmp['Rank'].mean(), inplace=True)
    
    df = pd.concat([df, tmp])

df = df.sort_values(by=['Date', 'SecuritiesCode'])
df

Empty DataFrame
Columns: [Date, SecuritiesCode, Open, High, Low, Close, Volume, Target]
Index: []


/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


,Date,SecuritiesCode,Open,High,Low,Close,Volume,Target,RowId,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Rank
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,0.000730,20170104_1301,1.0,NaN,False,898.0
1,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,0.012324,20170104_1332,1.0,NaN,False,325.0
2,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,0.006154,20170104_1333,1.0,NaN,False,567.0
3,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,0.011053,20170104_1376,1.0,NaN,False,364.0
4,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,0.003026,20170104_1377,1.0,NaN,False,750.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,0.034816,20211203_9990,1.0,NaN,False,580.0
2332527,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,0.025478,20211203_9991,1.0,NaN,False,1119.0
2332528,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,-0.004302,20211203_9993,1.0,NaN,False,1941.0
2332529,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,0.009098,20211203_9994,1.0,NaN,False,1768.0


In [7]:
df['SecuritiesCode'] = df['SecuritiesCode'].astype(int)
df['Volume'] = df['Volume'].astype(int)

In [8]:
df.isna().sum()

Date                      0
SecuritiesCode            0
Open                      0
High                      0
Low                       0
Close                     0
Volume                    0
Target                    0
RowId                     0
AdjustmentFactor          0
ExpectedDividend    2313666
SupervisionFlag           0
Rank                      0
dtype: int64

In [9]:
feature_cols = ["Date", "SecuritiesCode", "Open", "High", "Low", "Close", "Volume"]

def create_data(df) :    
    df = df[feature_cols].copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].dt.strftime("%Y%m%d").astype(int)
    return df

X_train = create_data(df)
y_train = df["Target"].copy()
# y_train = train_prices["Rank"].copy()
X_train.head()

,Date,SecuritiesCode,Open,High,Low,Close,Volume
0,20170104,1301,2734.0,2755.0,2730.0,2742.0,31400
1,20170104,1332,568.0,576.0,563.0,571.0,2798500
2,20170104,1333,3150.0,3210.0,3140.0,3210.0,270800
3,20170104,1376,1510.0,1550.0,1510.0,1550.0,11300
4,20170104,1377,3270.0,3350.0,3270.0,3330.0,150800


In [10]:
lgb_params = {
    "objective": "regression",
    'metric': 'rmse',
    "n_estimators" : 200,
    "num_leaves" : 60,
    "learning_rate" : 0.1,
    "random_seed" : 10}

In [11]:
model = LGBMRegressor(**lgb_params)
    
model.fit(X_train, y_train)

LGBMRegressor(metric='rmse', n_estimators=200, num_leaves=60,
              objective='regression', random_seed=10)

In [12]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    
    display(prices)    
    df = create_data(prices)
        
    sample_prediction["Prediction"] = model.predict(df)
    sample_prediction = sample_prediction.sort_values(by = "Prediction", ascending=False)
    sample_prediction['Rank'] = np.arange(0,2000)
    sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending=True)
    sample_prediction.drop(["Prediction"],axis=1)
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]].copy()
    env.predict(submission)
    display(submission)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


,Date,RowId,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
0,2021-12-06,20211206_1301,1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,NaN,False
1,2021-12-06,20211206_1332,1332,592.0,599.0,588.0,589.0,1360800,1.0,NaN,False
2,2021-12-06,20211206_1333,1333,2368.0,2388.0,2360.0,2377.0,125900,1.0,NaN,False
3,2021-12-06,20211206_1375,1375,1230.0,1239.0,1224.0,1224.0,81100,1.0,NaN,False
4,2021-12-06,20211206_1376,1376,1339.0,1372.0,1339.0,1351.0,6200,1.0,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...
1995,2021-12-06,20211206_9990,9990,528.0,531.0,516.0,517.0,65300,1.0,NaN,False
1996,2021-12-06,20211206_9991,9991,796.0,800.0,785.0,785.0,29100,1.0,NaN,False
1997,2021-12-06,20211206_9993,9993,1645.0,1653.0,1627.0,1627.0,6200,1.0,NaN,False
1998,2021-12-06,20211206_9994,9994,2394.0,2433.0,2393.0,2418.0,7800,1.0,NaN,False


,Date,SecuritiesCode,Rank
0,2021-12-06,1301,843
1,2021-12-06,1332,1500
2,2021-12-06,1333,830
3,2021-12-06,1375,787
4,2021-12-06,1376,258
...,...,...,...
1995,2021-12-06,9990,1522
1996,2021-12-06,9991,616
1997,2021-12-06,9993,677
1998,2021-12-06,9994,881


,Date,RowId,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
0,2021-12-07,20211207_1301,1301,2998.0,3065.0,2990.0,3065.0,19100,1.0,NaN,False
1,2021-12-07,20211207_1332,1332,569.0,569.0,535.0,556.0,6449200,1.0,NaN,False
2,2021-12-07,20211207_1333,1333,2382.0,2417.0,2371.0,2409.0,127300,1.0,NaN,False
3,2021-12-07,20211207_1375,1375,1227.0,1266.0,1227.0,1264.0,128600,1.0,NaN,False
4,2021-12-07,20211207_1376,1376,1374.0,1395.0,1366.0,1395.0,5800,1.0,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...
1995,2021-12-07,20211207_9990,9990,526.0,535.0,524.0,535.0,57800,1.0,NaN,False
1996,2021-12-07,20211207_9991,9991,795.0,806.0,792.0,805.0,48500,1.0,NaN,False
1997,2021-12-07,20211207_9993,9993,1640.0,1640.0,1620.0,1620.0,6600,1.0,NaN,False
1998,2021-12-07,20211207_9994,9994,2437.0,2440.0,2423.0,2440.0,5200,1.0,NaN,False


,Date,SecuritiesCode,Rank
0,2021-12-07,1301,900
1,2021-12-07,1332,1994
2,2021-12-07,1333,894
3,2021-12-07,1375,381
4,2021-12-07,1376,699
...,...,...,...
1995,2021-12-07,9990,1604
1996,2021-12-07,9991,1580
1997,2021-12-07,9993,734
1998,2021-12-07,9994,1391
